### Interacting with the SHEEP server

Assuming the sheep server is running, and the ```sheep_client.py``` module has the correct BASE_URL, we can configure and run SHEEP jobs using the commands in this notebook.

First lets just set up a directory path (we only need this so we can find the circuit file later on):

In [67]:
import os
if "SHEEP_HOME" in os.environ.keys():
  SHEEP_HOME = os.environ["SHEEP_HOME"]
else:
  SHEEP_HOME = os.path.join(os.environ["HOME"],"SHEEP","pysheep")
import sys
SHEEP_HOME = '/home/nitin/Desktop/sheepD/SHEEP/pysheep/'
sys.path.append(SHEEP_HOME)

In [68]:
from pysheep import interface
         

In [69]:
from pysheep.interface import sheep_client

In [70]:
sheep_client.new_job()


{'content': '', 'status_code': 200}

List the available contexts:

In [71]:
sheep_client.get_available_contexts()

{'content': ['Clear', 'HElib_F2', 'HElib_Fp', 'TFHE', 'SEAL', 'LP'],
 'status_code': 200}

Lets do a test with HElib_F2:

In [72]:
sheep_client.set_context("LP")

{'content': '', 'status_code': 200}

List the available input types:

In [73]:
sheep_client.get_available_input_types()

{'content': ['bool',
  'uint8_t',
  'uint16_t',
  'uint32_t',
  'int8_t',
  'int16_t',
  'int32_t'],
 'status_code': 200}

Let's use uint16_t:

In [74]:
sheep_client.set_input_type("int16_t")

{'content': '', 'status_code': 200}

In [75]:
sheep_client.get_parameters()

{'content': {'n': 256}, 'status_code': 200}

Now let's give it a circuit.  There are plenty of simple circuits in the ```benchmark_inputs/``` directory.  We need to provide the full path to the circuit though so we'll use the SHEEP_HOME we defined at the start.

In [76]:
r=sheep_client.set_circuit(os.path.join(SHEEP_HOME,"benchmark_inputs/low_level/circuits/circuit-ADD-1-plain.sheep"))

We can see the names of the "inputs" that this circuit expects:

In [77]:
sheep_client.get_inputs()

{'content': ['input_1', 'input_2'], 'status_code': 200}

In [78]:
sheep_client.set_inputs({'input_1':12, 'input_2':31})

{'input_1': 12, 'input_2': 31}


{'content': '', 'status_code': 200}

In [79]:
sheep_client.get_const_inputs()

{'content': ['N', 'N2'], 'status_code': 200}

In [80]:
sheep_client.set_const_inputs({"N":4, "N2":8})

{'N2': 8, 'N': 4}


{'content': '', 'status_code': 200}

Now we can set values for those inputs.  The circuit we've chosen just adds the two inputs together, so lets do 55+44 (and hope we get the answer 99 at the end!)

In [63]:
#sheep_client.set_inputs({"input_0":10,"input_1":12, "N":13})

We have set the context, the input_type, the circuit_file and the input values - we should be ready to go!  Let's print out the configuration, and then check that we are fully configured.

In [81]:
sheep_client.get_config()

{'content': {'circuit_filename': '',
  'context': 'LP',
  'eval_strategy': 'serial',
  'input_type': 'int16_t',
  'parameters': {'n': 256}},
 'status_code': 200}

In [82]:
sheep_client.is_configured()

http://localhost:34568/SheepServer/job/


{'content': {'job_configured': True}, 'status_code': 200}

OK, let's execute the test!

In [83]:
sheep_client.run_job()

http://localhost:34568/SheepServer/job/
200


{'content': '', 'status_code': 200}

The SHEEP server just used our selected "context" (i.e. HE library) to encrypt our inputs, evaluate our circuit, and decrypt the outputs.  It also then evaluated the same circuit "in the clear" (i.e. with no encryption) so that we can check we got the right answer.   The result, including the output values, the processing times, and the clear-text check, are available as a json object.

In [84]:
r=sheep_client.get_results()
from pprint import pprint
pprint(r)

{'content': {'cleartext check': {'is_correct': True},
             'outputs': {'output_0': '96'},
             'timings': {'decryption': '38.100000',
                         'encryption': '1155.497000',
                         'evaluation': '154.300000'}},
 'status_code': 200}


So we can see that 
 * we got the answer "99" (which is 55+44, the sum of our two inputs) 
 * all the output values we got (just one in this case) matched the result from the "cleartext" context.
 * from the timings (which are in microseconds), the encryption and decryption were fast, but evaluating the circuit took some time.